In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [30]:
prices = pd.read_csv('prices.csv')
wind = pd.read_csv('wind_power_cleaned.csv')
grid_excess = pd.read_csv('grid_excess.csv')

In [31]:
prices['date'] = pd.to_datetime(prices['date'])
wind['date'] = pd.to_datetime(wind['date'])

In [32]:
data = pd.merge(prices, wind, on=['date', 'hour'], how='outer')

#select every 18th day and save that to a new dataframe
data['date'] = pd.to_datetime(data['date'])
data

,date,hour,price,wind power
0,2023-01-01,0,2.01,114.776290
1,2023-01-01,1,1.38,117.265806
2,2023-01-01,2,0.09,117.016935
3,2023-01-01,3,0.08,113.002903
4,2023-01-01,4,0.05,108.146613
...,...,...,...,...
8756,2023-12-31,20,34.89,83.135484
8757,2023-12-31,21,29.60,72.667742
8758,2023-12-31,22,28.67,69.600000
8759,2023-12-31,23,26.87,72.667742


In [33]:
#fill nan values with the most recent value
data['wind power'] = data['wind power'].fillna(method='ffill')

C:\Users\janle\AppData\Local\Temp\ipykernel_14628\3486741652.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data['wind power'] = data['wind power'].fillna(method='ffill')


In [34]:
#get a list of every 18th day in 2023
selected_dates = pd.date_range(start='2023-01-01', end='2023-12-31', freq='19D')

In [35]:
#select every 18th day
scenarios_price = {}
scenarios_wind = {}
i=1
for date in selected_dates:
    scenarios_price[i] = prices[prices['date'] == date].reset_index(drop=True).drop(columns=['date', 'hour'])
    scenarios_wind[i] = wind[wind['date'] == date].reset_index(drop=True).drop(columns=['date', 'hour'])
    i+=1

In [36]:
#make 20 times 20 times 2 scenarios
scenarios = {}
i=1
for price in scenarios_price:
    for wind in scenarios_wind:
        for grid in grid_excess:
            df = pd.merge(scenarios_price[price], scenarios_wind[wind], left_index=True, right_index=True)
            #add column with grid excess
            df['grid_excess'] = grid_excess[grid]
            scenarios[i] = df
            i+=1

In [46]:
#make a dataframe with all scenarios
scenarios_df = pd.DataFrame()
for scenario in scenarios:
    scenarios_df = pd.concat([scenarios_df, scenarios[scenario]], axis=1)

In [47]:
scenarios_df

,price,wind power,grid_excess,price,wind power,grid_excess,price,wind power,grid_excess,price,...,grid_excess,price,wind power,grid_excess,price,wind power,grid_excess,price,wind power,grid_excess
0,2.01,114.776290,1,2.01,114.776290,0,2.01,114.776290,1,2.01,...,1,0.00,15.778419,1,0.00,15.778419,0,0.00,15.778419,1
1,1.38,117.265806,0,1.38,117.265806,0,1.38,117.265806,0,1.38,...,0,0.00,14.820387,0,0.00,14.820387,0,0.00,14.820387,0
2,0.09,117.016935,0,0.09,117.016935,1,0.09,117.016935,1,0.09,...,1,0.00,16.245968,0,0.00,16.245968,1,0.00,16.245968,1
3,0.08,113.002903,0,0.08,113.002903,1,0.08,113.002903,1,0.08,...,1,0.00,21.937903,0,0.00,21.937903,1,0.00,21.937903,1
4,0.05,108.146613,1,0.05,108.146613,1,0.05,108.146613,0,0.05,...,0,0.00,20.196032,1,0.00,20.196032,1,0.00,20.196032,0
5,0.00,109.434839,1,0.00,109.434839,0,0.00,109.434839,1,0.00,...,1,0.00,19.268613,1,0.00,19.268613,0,0.00,19.268613,1
6,0.00,104.349194,1,0.00,104.349194,1,0.00,104.349194,1,0.00,...,1,0.02,17.511322,1,0.02,17.511322,1,0.02,17.511322,1
7,0.00,103.625806,0,0.00,103.625806,0,0.00,103.625806,1,0.00,...,1,16.06,17.589597,0,16.06,17.589597,0,16.06,17.589597,1
8,0.00,103.569839,0,0.00,103.569839,0,0.00,103.569839,0,0.00,...,0,38.20,18.876742,0,38.20,18.876742,0,38.20,18.876742,0
9,0.04,104.884516,1,0.04,104.884516,1,0.04,104.884516,1,0.04,...,1,38.75,18.812710,1,38.75,18.812710,1,38.75,18.812710,1


In [48]:
#save as csv
scenarios_df.to_csv('scenarios.csv', index=False)